In [1]:
import torch
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.bfloat16, device_map="auto"
)
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

2025-06-30 12:08:10.772793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751285290.955612      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751285291.008168      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [26]:
def get_answer_from_model(task_text):
    sys_prompt = """
    You are a mathematician. You need to solve the task.
    """
    messages = [
        {
            "role": "system",
            "content": sys_prompt,
        },
        {
            "role": "user",
            "content": f"Answer only the final numeric result, nothing else. Task: {task_text}",
        }
    ]
    input_tensor = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    )
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=50)
    result = tokenizer.decode(
        outputs[0][input_tensor.shape[1] :], skip_special_tokens=True
    )

    cleaned = re.findall(r"-?\d+(?:\.\d+)?", result)
    if cleaned:
        extracted_answer = cleaned[0]
    else:
        extracted_answer = result.strip()
    return extracted_answer

In [27]:
df = pd.read_csv("/kaggle/input/ai-challenge-first/train.csv")

submission_rows = []
for idx, row in df.iterrows():
    task = row["task"]
    answer = get_answer_from_model(task)
    submission_rows.append({"task": task, "answer": f"[{answer}]"})

submission_df = pd.DataFrame(submission_rows)
submission_df.to_csv("submission.csv", index=False)